In [1]:
import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List, Tuple, NamedTuple, Callable
import scml
import mylib

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [3]:
%%time
df = pd.read_csv("input/validation_data.csv", engine="c", low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30108 entries, 0 to 30107
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   worker      30108 non-null  int64 
 1   less_toxic  30108 non-null  object
 2   more_toxic  30108 non-null  object
dtypes: int64(1), object(2)
memory usage: 705.8+ KB
Wall time: 227 ms


In [4]:
texts = set(df["less_toxic"]) | set(df["more_toxic"])
texts = list(texts)
df = pd.DataFrame(data={"text": texts})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14251 non-null  object
dtypes: object(1)
memory usage: 111.5+ KB


# Preprocess Text
Speed dropped from 1200 to 1000 it/s

In [5]:
def pre1(row) -> str:
    return mylib.pre1(row["text"])


def pre2(row) -> str:
    return mylib.pre2(row["text1"])


col = "text1"
print(col)
df[col] = df.progress_apply(pre1, axis=1)
col = "text2"
print(col)
df[col] = df.progress_apply(pre2, axis=1)

text1


100%|████████████████████████████████████████| 14251/14251 [00:50<00:00, 282.73it/s]


text2


100%|█████████████████████████████████████████| 14251/14251 [05:53<00:00, 40.35it/s]


In [13]:
cols = ["text", "text1", "text2"]
df[cols].sample(10).head(10)

,text,text1,text2
11851,"""\n\nAnd western reports are? You're a biased fuck. So, the US, UK, or France can claim whatever casualties they want... thats fine, but when Libya claims to have caused casuatlies you immediately lable it """"unreliable"""" and remove it? Your bias is showing asswipe... cover it up before you stain Wikipedia's name even more.- AGSman61""",""" And western reports are? You're a biased fuck. So, the US, UK, or France can claim whatever casualties they want... thats fine, but when Libya claims to have caused casuatlies you immediately lable it """"unreliable"""" and remove it? Your bias is showing asswipe... cover it up before you stain Wikipedia's name even more.- AGSman61""",""" And western reports are? you are a biased fuck. So, the US, UK, or France can claim whatever casualties they want... thats fine, but when Libya claims to have caused casuatlies you immediately lable it """"unreliable"""" and remove it? Your bias is showing asswipe... cover it up before you stain Wikipedia's name even more.- AGSman61"""
7947,"""""""Rock critics"""" are morons. Everyone knows that. However, there is no reason to be """"ashamed"""" here, especially when the statement is followed by evidence for record sales of 300 million albums and proof that Queen is outselling The Beatles in the UK. Everyone reading it going to see right away that the critics (who like idiotic garage bands that only know three chords) must be wrong. At the same time, it is important that this page does not read too much like a fan page. That is why that portion of the sentence is there. Feel free to make it sound less harsh though. \n\n""","""""""Rock critics"""" are morons. Everyone knows that. However, there is no reason to be """"ashamed"""" here, especially when the statement is followed by evidence for record sales of 300 million albums and proof that Queen is outselling The Beatles in the UK. Everyone reading it going to see right away that the critics (who like idiotic garage bands that only know three chords) must be wrong. At the same time, it is important that this page does not read too much like a fan page. That is why that portion of the sentence is there. Feel free to make it sound less harsh though. ""","""""""Rock critics"""" are morons. Everyone knows that. However, there is no reason to be """"ashamed"""" here, especially when the statement is followed by evidence for record sales of 300 million albums and proof that Queen is outselling The Beatles in the UK. Everyone reading it going to see right away that the critics (who like idiotic garage bands that only know three chords) must be wrong. At the same time, it is important that this page does not read too much like a fan page. That is why that portion of the sentence is there. Feel free to make it sound less harsh though. """
3177,"FUCK YOU THUE \n\nYOU BRAIN DEAD FUCK, YOU CAN'T BLOCK ME. THANKS TO PROXY SERVERS AND ANON BROWSING I CAN COME TO WIKI ZILLION TIMES SO IF SCUM LIKE YOU THINK THAT YOU CAN BLOCK ME THEN YOU ARE IN FOR BIG SURPRISE YOU FAGGOT. I AM GIVING YOU LAST CHANCE IN CIVILIZED WAY TO STOP VANDALIZING KASH JAFFREY. DON'T FUCK WITH CANADIAN CONTENT, ITS TO ADVANCE FOR YOUR EMPTY BRAIN TO CONCIEVE. AND LASTLY IF YOU WANT TO LEAVE MESSAGE THEN WRITE (DON'T USE PREDEFINE ONE LINERS.) SEE YA ASSHOLE\n\nJOEY","FUCK YOU THUE YOU BRAIN DEAD FUCK, YOU CAN'T BLOCK ME. THANKS TO PROXY SERVERS AND ANON BROWSING I CAN COME TO WIKI ZILLION TIMES SO IF SCUM LIKE YOU THINK THAT YOU CAN BLOCK ME THEN YOU ARE IN FOR BIG SURPRISE YOU FAGGOT. I AM GIVING YOU LAST CHANCE IN CIVILIZED WAY TO STOP VANDALIZING KASH JAFFREY. DON'T FUCK WITH CANADIAN CONTENT, ITS TO ADVANCE FOR YOUR EMPTY BRAIN TO CONCIEVE. AND LASTLY IF YOU WANT TO LEAVE MESSAGE THEN WRITE (DON'T USE PREDEFINE ONE LINERS.) SEE YA ASSHOLE JOEY","FUCK YOU THUE YOU BRAIN DEAD FUCK, YOU cannot BLOCK ME. THANKS TO PROXY SERVERS AND ANON BROWSING I CAN COME TO [a website or database developed collaboratively by an online

In [7]:
df[cols].sample(10).head(10)

text  \
4865                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [11]:
df[cols].sample(10).head(10)

text  \
11104                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

# Review data

In [9]:
df[cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14251 non-null  object
 1   text1   14251 non-null  object
 2   text2   14251 non-null  object
dtypes: object(3)
memory usage: 334.1+ KB


In [10]:
%%time
df[cols].to_parquet("output/pre_val.parquet", index=False)

Wall time: 112 ms
